In [1]:
# import sys
# !{sys.executable} -m pip install Jinja2
import glacierml as gl
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
parameterization = '4'
method = '1'

2023-08-07 12:22:28.980642: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-07 12:22:29.025677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 12:22:29.620772: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
def stats_builder(st1 = 1e-5, st2 = 1e4):
    
#     boundary = [(str(st1), str(st2))]
    
    col_list = [ 'Area', 
#                 'Slope','Lmax','Zmin',
#                 'Zmax',
                'WT1','Thick Diff',
                'Vol Diff'
#                'Farinotti Mean Thickness'
               ]
    name = ['mean', 'median', 'min', 'max','IQR','STD','count']

    df = pd.DataFrame( columns = col_list, index = name)
    data = gl.load_notebook_data('4','first')
    x = np.array(data['FMT'])
    y = np.array(data['WT1'])
    area = np.array(data['Area'])
    
    vol_f = (x / 1e3) * area
    vol = (y / 1e3) * area
    
    thick_f = vol_f / area * 1e3
    thick = vol / area * 1e3
    
    vol_diff = vol - vol_f
    data['Thick Diff'] = thick - thick_f
    data['Vol Diff'] = vol_diff
    data = data.iloc[np.where(
        (vol > st1) & (vol < st2)
    )]
    vol_sub = vol[np.where(
        (vol > st1) & (vol < st2)
    )]
    feat_list = col_list
    for i in feat_list:
        dft = data[i]
        upp = np.nanquantile(dft, 0.75)
        low = np.nanquantile(dft, 0.25)
#         print(np.quantile(dft, 0.75))
#         print(np.quantile(dft, 0.25))
        functions = [
            np.round(np.nanmean(dft), 3),
            np.round(np.nanmedian(dft), 3), 
            np.round(np.nanmin(dft), 3),
            np.round(np.nanmax(dft), 3),
            np.round(upp - low, 3),
            np.round(np.nanstd(dft),3),
            len(dft)
        ]
        for n, fn in zip(name, functions):
            df[i].loc[n] = fn
    df = df.rename(columns = {
        'Area':'Area (km$^2$)',
        'Slope':'Slope (deg)',
        'Lmax':'Max Length (m)',
        'Zmin':'Min Elevation (m)',
        'WT1':'Est Thick (m)',
        'Vol Diff':'Vol Diff (km$^3$)'
#         'Farinotti Mean Thickness':'Farinotti Thickness'
    })
    df = df.round(decimals = 3)
    return df, data, vol, vol_sub

In [9]:
stats, data, vol, vol_sub = stats_builder(1e-5, 1e4)

In [10]:
stats

,Area (km$^2$),Est Thick (m),Thick Diff,Vol Diff (km$^3$)
mean,3.446,49.575,21.258,-0.138
median,0.252,46.0,24.0,0.005
min,0.01,16.0,-433.0,-557.26
max,7537.579,819.0,270.0,816.781
IQR,0.689,14.0,14.0,0.01
STD,51.081,17.307,18.654,5.382
count,216501,216501,216501,216501


In [ ]:
print(stats.to_latex(float_format="%.2f"))

In [ ]:
data

In [ ]:
data = data.dropna(subset = 'Farinotti Mean Thickness')

In [ ]:
import matplotlib.pyplot as plt




In [ ]:
np.sqrt(55000)

In [ ]:
plt.hist(data['Vol Diff'],75)
plt.yscale('log')

In [ ]:
f = data['Vol Diff']
plt.hist((f), 25)
plt.yscale('log')
plt.plot(
    [np.percentile(f,50),np.percentile(f,50)],
    [0,1e4], 'k--'
)
plt.plot(
    [np.percentile(f,25),np.percentile(f,25)],
    [0,1e4], 'k--'
)
plt.plot(
    [np.percentile(f,75),np.percentile(f,75)],
    [0,1e4], 'k--'
)
# plt.text(
    
    
#     (f).mean() - 0.5,
#     955, s = 'Median'
# )
# plt.text(
#     np.percentile(f,75) + .18 ,
#     900, s = 'Q3'
# )
# plt.text(
#     np.percentile(f, 25) - 0.76,
#     900, s = 'Q1'
# )

In [ ]:
data

In [ ]:
data['Thick Diff'] = data['Weighted Mean Thickness 1'] - data['Farinotti Mean Thickness']
data

In [ ]:
f = data['Thick Diff']
plt.hist(data['Thick Diff'], 25)
plt.yscale('log')
plt.plot(
    [np.percentile(f,50),np.percentile(f,50)],
    [0,1e5], 'k--'
)
plt.plot(
    [np.percentile(f,25),np.percentile(f,25)],
    [0,1e5], 'k--'
)
plt.plot(
    [np.percentile(f,75),np.percentile(f,75)],
    [0,1e5], 'k--'
)

In [ ]:
np.percentile(f,50)

In [ ]:
x = data['Farinotti Mean Thickness'] / 1e3 * data['Area']
y = data['Weighted Mean Thickness 1'] / 1e3 * data['Area']

np.nanmean(x - y)


In [ ]:
len(data)

In [ ]:
(len(data) / 216501) * 100

In [ ]:
(sum(vol_sub) / sum(vol)) * 100

In [ ]:
sum(vol) / 1e3

In [ ]:
sum(vol_sub) / 1e3